# Clean dim_order_pages từ Silver lên Gold

## Mục tiêu
- Load dữ liệu từ Silver.dim_order_pages → Gold.dim_order_pages
- Clean và chuẩn hóa dữ liệu cho EDA
- Đảm bảo data quality cho Gold layer

## Quy trình 6 bước
1. **Load from Silver** - Đọc dữ liệu từ Silver database
2. **Data Quality Check** - Phân tích chất lượng dữ liệu
3. **Data Cleaning** - Xử lý các vấn đề về dữ liệu
4. **Data Validation** - Kiểm tra kết quả sau khi clean
5. **Load to Gold** - Load vào Gold database
6. **Create Dictionary** - Tạo data dictionary


## Bước 1: Import và Setup


In [5]:
# Import các thư viện cần thiết
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Load biến môi trường từ file .env
load_dotenv()

# Lấy thông tin kết nối database
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_SILVER = os.getenv("DB_SILVER")
DB_GOLD = os.getenv("DB_GOLD")

# Tạo kết nối tới Silver và Gold database
silver_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_SILVER}")
gold_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_GOLD}")

print("Đã kết nối Silver và Gold database thành công")
print(f"Silver DB: {DB_SILVER}")
print(f"Gold DB: {DB_GOLD}")
print(f"Thời gian bắt đầu: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Đã kết nối Silver và Gold database thành công
Silver DB: winner_silver
Gold DB: winner_gold
Thời gian bắt đầu: 2025-10-17 23:24:58


## Bước 2: Load dữ liệu từ Silver


In [6]:
# Load dữ liệu dim_order_pages từ Silver database
print("=== LOADING DIM_ORDER_PAGES FROM SILVER ===")

# Load toàn bộ bảng dim_order_pages
pages_df = pd.read_sql("SELECT * FROM dim_order_pages", silver_engine)

# Hiển thị thông tin cơ bản về dataset
print(f"Shape: {pages_df.shape}")
print(f"Columns: {list(pages_df.columns)}")
print(f"Memory usage: {pages_df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

# Hiển thị sample data
print("\n=== SAMPLE DATA ===")
print(pages_df.head(3))

# Hiển thị data types
print("\n=== DATA TYPES ===")
print(pages_df.dtypes)


=== LOADING DIM_ORDER_PAGES FROM SILVER ===
Shape: (74, 4)
Columns: ['page_id', 'page_name', 'page_username', 'page_created_at']
Memory usage: 0.01 MB

=== SAMPLE DATA ===
           page_id        page_name page_username     page_created_at
0  519409817926702  Gentleman Store          None 2025-01-08 15:15:16
1  490071977529040       LiLi Store          None 2025-01-08 15:14:52
2  503815352816773     G&T Clothing          None 2024-12-25 14:03:02

=== DATA TYPES ===
page_id                    object
page_name                  object
page_username              object
page_created_at    datetime64[ns]
dtype: object


## Bước 3: Data Quality Check


In [7]:
# Phân tích chất lượng dữ liệu
print("=== DATA QUALITY ANALYSIS ===")

# 1. Null ratio analysis
print("\n1. NULL RATIO ANALYSIS")
null_ratio = pages_df.isnull().mean().sort_values(ascending=False)
print("Tỷ lệ null theo cột:")
for col, ratio in null_ratio.items():
    print(f"  {col}: {ratio:.3f} ({ratio*100:.1f}%)")

# 2. Duplicate check
print(f"\n2. DUPLICATE CHECK")
if 'page_id' in pages_df.columns:
    dup_count = pages_df["page_id"].duplicated().sum()
    print(f"Số lượng page_id trùng lặp: {dup_count}")

# 3. Value analysis cho các trường quan trọng
print(f"\n3. VALUE ANALYSIS")

# String fields analysis
string_fields = ['page_name', 'page_url', 'page_type']
for field in string_fields:
    if field in pages_df.columns:
        print(f"\n--- {field.upper()} ANALYSIS ---")
        value_counts = pages_df[field].value_counts(dropna=False).head(10)
        print(value_counts)
        print(f"Null count: {pages_df[field].isnull().sum()}")

# 4. Overall quality summary
print(f"\n4. OVERALL QUALITY SUMMARY")
total_columns = len(pages_df.columns)
high_null_columns = len(null_ratio[null_ratio > 0.7])
print(f"Total columns: {total_columns}")
print(f"Columns with >70% null: {high_null_columns}")
print(f"Data quality score: {((total_columns - high_null_columns) / total_columns * 100):.1f}%")


=== DATA QUALITY ANALYSIS ===

1. NULL RATIO ANALYSIS
Tỷ lệ null theo cột:
  page_username: 0.554 (55.4%)
  page_id: 0.014 (1.4%)
  page_name: 0.014 (1.4%)
  page_created_at: 0.000 (0.0%)

2. DUPLICATE CHECK
Số lượng page_id trùng lặp: 0

3. VALUE ANALYSIS

--- PAGE_NAME ANALYSIS ---
page_name
Xưởng May Kim Anh           2
LiLi Store                  1
G&T Clothing                1
Yone36 - Hàng Xuất Dư       1
Quần áo nam cao cấp         1
Soái Ca Closet              1
Sùn Hàng Hiệu Oulet         1
Huyên Store                 1
Gentleman Store             1
Thanh lý hàng hiệu UK,US    1
Name: count, dtype: int64
Null count: 1

4. OVERALL QUALITY SUMMARY
Total columns: 4
Columns with >70% null: 0
Data quality score: 100.0%


## Bước 4: Data Cleaning


In [8]:
# Xử lý cột page_username
print("=== CLEANING PAGE_USERNAME COLUMN ===")

# Tạo bản copy để xử lý
pages_clean = pages_df.copy()

# Kiểm tra cột page_username
if 'page_username' in pages_clean.columns:
    print("Trạng thái cột page_username:")
    print(f"Null count: {pages_clean['page_username'].isnull().sum()}")
    print(f"Null percentage: {pages_clean['page_username'].isnull().mean() * 100:.1f}%")
    
    # Xóa cột page_username
    print("\nXóa cột page_username...")
    pages_clean = pages_clean.drop(columns=['page_username'])
    print("Đã xóa cột page_username thành công")
else:
    print("Cột page_username không tồn tại trong dataset")

# Hiển thị kết quả
print(f"\nShape sau khi xóa: {pages_clean.shape}")
print(f"Columns còn lại: {list(pages_clean.columns)}")


=== CLEANING PAGE_USERNAME COLUMN ===
Trạng thái cột page_username:
Null count: 41
Null percentage: 55.4%

Xóa cột page_username...
Đã xóa cột page_username thành công

Shape sau khi xóa: (74, 3)
Columns còn lại: ['page_id', 'page_name', 'page_created_at']


## Bước 5: Load vào Gold Database


In [10]:
# Load vào Gold Database
print("=== LOADING TO GOLD DATABASE ===")

from sqlalchemy.types import BigInteger, Integer, String, Text, DateTime

# Định nghĩa schema mapping cho MySQL
dtype_mapping = {}

# ID fields - String (vì page_id có thể chứa string như 'lv_230361475')
id_fields = ["page_id", "order_id"]
for field in id_fields:
    if field in pages_clean.columns:
        dtype_mapping[field] = String(100)  # Thay đổi từ BigInteger sang String

# String fields - VARCHAR(255)
string_fields = ["page_name", "page_type"]
for field in string_fields:
    if field in pages_clean.columns:
        dtype_mapping[field] = String(255)

# Text fields - Text()
text_fields = ["page_url"]
for field in text_fields:
    if field in pages_clean.columns:
        dtype_mapping[field] = Text()

# DateTime fields
datetime_fields = ["created_at", "updated_at", "page_created_at"]
for field in datetime_fields:
    if field in pages_clean.columns:
        dtype_mapping[field] = DateTime()

print(f"Schema mapping defined for {len(dtype_mapping)} columns")
print("Updated page_id and order_id to String type to handle mixed data types")

# Load vào Gold database
table_name = "dim_order_pages"

try:
    pages_clean.to_sql(
        table_name,
        con=gold_engine,
        if_exists="replace",  # Ghi đè dữ liệu cũ
        index=False,
        dtype=dtype_mapping
    )
    
    print(f"Đã load {pages_clean.shape[0]} records vào Gold: {table_name}")
    
    # Verify load
    verification_query = f"SELECT COUNT(*) as count FROM {table_name}"
    verification_result = pd.read_sql(verification_query, gold_engine)
    loaded_count = verification_result['count'].iloc[0]
    
    print(f"Verification: {loaded_count} records trong database")
    print(f"Schema: {len(dtype_mapping)} columns với explicit typing")
    
    # Sample data từ database
    sample_query = f"SELECT * FROM {table_name} LIMIT 5"
    sample_db = pd.read_sql(sample_query, gold_engine)
    print(f"\nSample data từ Gold database:\n{sample_db}")
    
except Exception as e:
    print(f"Error loading to Gold: {str(e)}")
    raise

print(f"\nLoad dim_order_pages completed successfully!")
print(f"Ready for EDA and analytics")


=== LOADING TO GOLD DATABASE ===
Schema mapping defined for 3 columns
Updated page_id and order_id to String type to handle mixed data types
Đã load 74 records vào Gold: dim_order_pages
Verification: 74 records trong database
Schema: 3 columns với explicit typing

Sample data từ Gold database:
           page_id              page_name     page_created_at
0  519409817926702        Gentleman Store 2025-01-08 15:15:16
1  490071977529040             LiLi Store 2025-01-08 15:14:52
2  503815352816773           G&T Clothing 2024-12-25 14:03:02
3  500659586470191  Yone36 - Hàng Xuất Dư 2024-12-24 13:26:21
4  535651999625592    Quần áo nam cao cấp 2024-12-24 13:23:57

Load dim_order_pages completed successfully!
Ready for EDA and analytics


## Bước 6: Tạo Data Dictionary cho Gold


In [11]:
# Tạo Data Dictionary cho Gold
print("=== CREATING GOLD DATA DICTIONARY ===")

def get_business_meaning_vietnamese(column_name):
    """Get business meaning for each column in Vietnamese"""
    business_meanings = {
        # Định danh & Cơ bản
        "page_id": "Mã định danh trang duy nhất (khóa chính)",
        "order_id": "Mã định danh đơn hàng (khóa ngoại đến fact_orders)",
        
        # Thông tin trang
        "page_name": "Tên trang web/ứng dụng",
        "page_url": "URL đầy đủ của trang",
        "page_type": "Loại trang (web, mobile, app, etc.)",
        
        # Thời gian
        "created_at": "Thời gian tạo bản ghi",
        "updated_at": "Thời gian cập nhật bản ghi cuối cùng"
    }
    return business_meanings.get(column_name, "Chưa định nghĩa ý nghĩa business")

# Tạo Data Dictionary
dict_data = []
for col in pages_clean.columns:
    col_info = {
        "table_name": "dim_order_pages",
        "column_name": col,
        "dtype": str(pages_clean[col].dtype),
        "sql_type": str(dtype_mapping.get(col, "Not defined")),
        "null_count": pages_clean[col].isnull().sum(),
        "null_pct": round(pages_clean[col].isnull().mean() * 100, 2),
        "unique_count": pages_clean[col].nunique(),
        "sample_values": str(pages_clean[col].dropna().unique()[:3].tolist()),
        "business_meaning": get_business_meaning_vietnamese(col),
        "extraction_date": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    dict_data.append(col_info)

data_dictionary = pd.DataFrame(dict_data)

# Hiển thị Data Dictionary
print(f"Generated Data Dictionary for {len(data_dictionary)} columns")
print("\n=== DATA DICTIONARY ===")
print(data_dictionary)

# Lưu vào file Excel
excel_path = "Technical_Document/Gold_dictionary.xlsx"

try:
    from openpyxl import load_workbook
    
    # Kiểm tra file Excel có tồn tại không
    try:
        # Load workbook hiện tại
        wb = load_workbook(excel_path)
        
        # Lấy sheet đầu tiên
        ws = wb.active
        
        # Kiểm tra xem có dữ liệu cũ không
        if ws.max_row > 1:
            print(f"Found existing data in {excel_path}, appending new data...")
        else:
            print(f"File {excel_path} exists but is empty, adding header and data...")
            
    except FileNotFoundError:
        print(f"File {excel_path} not found, creating new file...")
        wb = None
    except Exception as e:
        print(f"Error loading {excel_path}: {str(e)}, creating new file...")
        wb = None
    
    if wb is None:
        # Tạo file mới với header
        data_dictionary.to_excel(excel_path, index=False, sheet_name='Gold_Dictionary')
        print(f"✅ Created new file: {excel_path}")
    else:
        # Append vào file hiện tại
        from openpyxl.utils.dataframe import dataframe_to_rows
        
        # Tìm dòng cuối cùng có dữ liệu
        last_row = ws.max_row
        
        # Thêm dữ liệu mới từ dòng tiếp theo
        for r in dataframe_to_rows(data_dictionary, index=False, header=False):
            last_row += 1
            for c_idx, value in enumerate(r, 1):
                ws.cell(row=last_row, column=c_idx, value=value)
        
        # Lưu file
        wb.save(excel_path)
        print(f"✅ Appended {len(data_dictionary)} rows to: {excel_path}")
    
except Exception as e:
    print(f"❌ Error appending to Data Dictionary: {str(e)}")
    # Fallback: tạo file mới
    try:
        data_dictionary.to_excel(excel_path, index=False)
        print(f"✅ Created new file as fallback: {excel_path}")
    except Exception as e2:
        print(f"❌ Error creating fallback file: {str(e2)}")

# Summary Report
print(f"\n=== GOLD DICTIONARY SUMMARY ===")
print(f"Table: dim_order_pages")
print(f"Total columns: {len(data_dictionary)}")
print(f"Data Dictionary: {excel_path}")
print(f"Created at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n=== SAMPLE DICTIONARY ENTRIES ===")
sample_dict = data_dictionary[['column_name', 'dtype', 'business_meaning']].head(5)
print(sample_dict)


=== CREATING GOLD DATA DICTIONARY ===
Generated Data Dictionary for 3 columns

=== DATA DICTIONARY ===
        table_name      column_name           dtype      sql_type  null_count  \
0  dim_order_pages          page_id          object  VARCHAR(100)           1   
1  dim_order_pages        page_name          object  VARCHAR(255)           1   
2  dim_order_pages  page_created_at  datetime64[ns]      DATETIME           0   

   null_pct  unique_count                                      sample_values  \
0      1.35            73  ['519409817926702', '490071977529040', '503815...   
1      1.35            72  ['Gentleman Store', 'LiLi Store', 'G&T Clothing']   
2      0.00            73  [Timestamp('2025-01-08 15:15:16'), Timestamp('...   

                           business_meaning      extraction_date  
0  Mã định danh trang duy nhất (khóa chính)  2025-10-17 23:26:20  
1                    Tên trang web/ứng dụng  2025-10-17 23:26:20  
2          Chưa định nghĩa ý nghĩa business  2025-